In [ ]:
import re 
# from pdfminer.high_level import extract_text
import PyPDF2
import os
from pathlib import Path
import pandas as pd
import glob
import pycountry
import time
from datetime import datetime
import shutil

import pymupdf4llm             

In [ ]:
def extract_text_from_pdfs(folder_path, text_folder_path):
    # Loop through all the files in the folder
    i = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            print(pdf_path)
            # Extract text from the PDF using pymupdf4llm.to_markdown()
            md_text = pymupdf4llm.to_markdown(pdf_path)

            # clean the text by deleting extra \n
            # cleaned_md_text = re.sub(r'\n\n', '\n', md_text)
            
            # Save the extracted text to a .txt file
            text_filename = os.path.splitext(filename)[0] + ".txt"
            text_file_path = os.path.join(text_folder_path, text_filename)
            
            with open(text_file_path, "w", encoding="utf-8") as text_file:
                text_file.write(md_text)
            
            print(f"Extracted text from {filename} to {text_filename}")
        



In [ ]:

def extract_text_from_pdf(pdf_path, text_folder_path):
    md_text = pymupdf4llm.to_markdown(pdf_path)

    # clean the text by deleting extra \n
    # cleaned_md_text = re.sub(r'\n\n', '\n', md_text)
    
    # Save the extracted text to a .txt file
    filename = os.path.basename(pdf_path)
    text_filename = os.path.splitext(filename)[0] + ".txt"
    text_file_path = os.path.join(text_folder_path, text_filename)
    
    with open(text_file_path, "w", encoding="utf-8") as text_file:
        text_file.write(md_text)
    
    print(f"Extracted text from {filename} to {text_filename}")

    return text_file_path




In [ ]:
def separate_section(text, text_list):
    original_list = [[] for i in range(7)]
    # remove/split explanatory notes (repeated probable associates)
    split_text = re.split(r'(EXPLANATORY NOTES)', text, maxsplit=1)
    new_text = split_text[0]
    break_pattern = r'(COMPANY INFORMATION|SHARE CAPITAL|PROBABLE ASSOCIATES|CURRENT APPOINTMENT HOLDERS|CURRENT SHAREHOLDERS|PAST APPOINTMENT HOLDERS|PAST SHAREHOLDERS|CURRENT CHARGES|PAST CHARGES|EXPLANATORY NOTES)'
    sections = re.split(break_pattern, new_text)
    sections = [section.strip() for section in sections if section.strip()]

    for i in range(len(sections)):
        section = sections[i]
        if i % 2 == 0:
            j = int(i/2 - 1)
            # print(f'one section:\n {section} \n finsihed! \n' )
            if -1 < j <7:
                original_list[j].append(section)
                # print(f'{j}th section:\n{section}\n')
                if -1 < j < 3:
                    text_list[j].append(section)
            # print(f'{j}th of textlist:\n {section}')

    part3_1 = part3_2 = part4_1 = part4_2 = part5_1 = part5_2 = ''
    pattern = r'Disclosure(.*?)Date(.*)'
    match = re.search(pattern, original_list[3][-1], re.DOTALL)
    if match:
        part3_1 = match.group(2)
        # print(f'part3_1:{part3_1}')
    
    pattern = r'(.*?)Name(.*?)Disclosure(.*?)Date(.*)'
    match = re.search(pattern, original_list[4][-1], re.DOTALL) 
    if match:
        part3_2 = match.group(1).strip() if match.group(1) else print("Part 3_2 not found")
        part4_1 = match.group(4).strip() if match.group(4) else print("Part 4_1 not found")
        # print(f'part4_1:{part4_1}')
    
    pattern = r'(.*?)Name(.*?)Disclosure(.*?)Date(.*)'
    match = re.search(pattern, original_list[5][-1], re.DOTALL)
    if match:
        part4_2 = match.group(1).strip() if match.group(1) else print("Part 4_2 not found")
        part5_1 = match.group(4).strip() if match.group(4) else print("Part 5_1 not found")
        # print(f'part5_1:{part5_1}')

    pattern = r'(.*?)Name(.*?)Disclosure(.*?)Date(.*)'
    match = re.search(pattern, original_list[6][-1], re.DOTALL)
    if match:
        part5_2 = match.group(1).strip() if match.group(1) else print("Part 5_2 not found")

    part3 = part3_1 + '\n' + part3_2
    part4 = part4_1 + '\n' + part4_2
    part5 = part5_1 + '\n' + part5_2

    text_list[3].append(part3.strip())
    text_list[4].append(part4.strip())
    text_list[5].append(part5.strip())
    

    return text_list


In [ ]:
def txt_from_pdf(folder_path, text_folder_path):
    file_list = []
    company_info_list = []
    share_captical_list = []
    prob_associates_list = []
    curr_appt_holder_list = []
    curr_share_holder_list = []
    past_appt_holder_list = []
    text_list = [company_info_list, share_captical_list, prob_associates_list, curr_appt_holder_list, curr_share_holder_list, past_appt_holder_list]

    i = 0
    for filename in os.listdir(folder_path):
        i += 1
        if filename.endswith(".pdf"):
            file_list.append(filename)
            pdf_path = os.path.join(folder_path, filename)
            text_file_path = extract_text_from_pdf(pdf_path, text_folder_path)

            with open(text_file_path, "r", encoding="utf-8") as file:
                txt = file.read()

            # remove the page front
            txt = re.sub('#### ENTITY PROFILE REPORT', '', txt)
            lines = txt.split('\n') 
            third_line = lines[2]
            txt = re.sub('\n'+third_line, '', txt)
            txt = re.sub(r'\n{3,}', '\n', txt)

            with open(text_file_path, "w", encoding="utf-8") as file:
                file.write(txt)

            # cannot separate sections: as long as there is heading, it is missed
            # seems like things before ### CURRENT APPOINTMENT HOLDERS is normal
            text_list = separate_section(txt, text_list)

        print(f'Finish {i}th file {filename}!\n')

        # trail
        # if i > 3:
        #     break

    return text_list, file_list
            
            

In [ ]:

folder_path = "1st 1000"
text_folder_path = 'new_txt'
text_list, file_list = txt_from_pdf(folder_path,text_folder_path)

In [ ]:
import pickle
with open('text_list.pkl', 'wb') as file:
    pickle.dump(text_list, file)

with open('file_list.pkl','wb') as file:
    pickle.dump(file_list, file)

In [ ]:
with open('text_list.pkl', 'rb') as file:
    text_list = pickle.load(file)
    
print(text_list[3][4])

# Separate trial

In [ ]:
company_info_list = []
share_captical_list = []
prob_associates_list = []
curr_appt_holder_list = []
curr_share_holder_list = []
past_appt_holder_list = []
past_share_holder_list = []
text_list = [company_info_list, share_captical_list, prob_associates_list, curr_appt_holder_list, curr_share_holder_list, past_appt_holder_list]

original_list =[[] for i in range(7)]
print(original_list)
def separate_section(text, file_list):
    original_list = [[] for i in range(7)]
    # remove/split explanatory notes (repeated probable associates)
    split_text = re.split(r'(EXPLANATORY NOTES)', text, maxsplit=1)
    new_text = split_text[0]
    break_pattern = r'(COMPANY INFORMATION|SHARE CAPITAL|PROBABLE ASSOCIATES|CURRENT APPOINTMENT HOLDERS|CURRENT SHAREHOLDERS|PAST APPOINTMENT HOLDERS|PAST SHAREHOLDERS|CURRENT CHARGES|PAST CHARGES|EXPLANATORY NOTES)'
    sections = re.split(break_pattern, new_text)
    sections = [section.strip() for section in sections if section.strip()]

    for i in range(len(sections)):
        section = sections[i]
        if i % 2 == 0:
            j = int(i/2 - 1)
            # print(f'one section:\n {section} \n finsihed! \n' )
            if -1 < j <7:
                original_list[j].append(section)
                # print(f'{j}th section:\n{section}\n')
                if -1 < j < 3:
                    file_list[j].append(section)
            # print(f'{j}th of textlist:\n {section}')

    # re-organize 
    # CURRENT APPOINTMENT HOLDER (index = 3)

    pattern = r'Disclosure(.*?)Date(.*)'
    match = re.search(pattern, original_list[3][-1], re.DOTALL)
    if match:
        part3_1 = match.group(2)
        # print(f'part3_1:{part3_1}')
    
    pattern = r'(.*?)Name(.*?)Disclosure(.*?)Date(.*)'
    match = re.search(pattern, original_list[4][-1], re.DOTALL) 
    if match:
        part3_2 = match.group(1).strip() if match.group(1) else print("Part 3_2 not found")
        part4_1 = match.group(4).strip() if match.group(4) else print("Part 4_1 not found")
        # print(f'part4_1:{part4_1}')
    
    pattern = r'(.*?)Name(.*?)Disclosure(.*?)Date(.*)'
    match = re.search(pattern, original_list[5][-1], re.DOTALL)
    if match:
        part4_2 = match.group(1).strip() if match.group(1) else print("Part 4_2 not found")
        part5_1 = match.group(4).strip() if match.group(4) else print("Part 5_1 not found")
        # print(f'part5_1:{part5_1}')

    pattern = r'(.*?)Name(.*?)Disclosure(.*?)Date(.*)'
    match = re.search(pattern, original_list[6][-1], re.DOTALL)
    if match:
        part5_2 = match.group(1).strip() if match.group(1) else print("Part 5_2 not found")

    part3 = part3_1 + '\n' + part3_2
    part4 = part4_1 + '\n' + part4_2
    part5 = part5_1 + '\n' + part5_2

    file_list[3].append(part3.strip())
    file_list[4].append(part4.strip())
    file_list[5].append(part5.strip())
    

    return file_list


## Sub functions

In [ ]:
# preprocessing: remove **, Entity Report
def get_first_and_last_non_signal_index(text):
    first_match = re.search(r'[A-Za-z0-9]', text)
    last_match = None
    for match in re.finditer(r'[A-Za-z0-9]', text):
        last_match = match
    first_index = first_match.start() if first_match else None
    last_index = last_match.start() if last_match else None
    
    return first_index, last_index


def clean_3(text):
    text = re.sub(r'#{2,}', r'\n', text)
    text = re.sub(r'\*\*ENTITY PROFILE REPORT\*\*', r'\n', text)
    text = re.sub(r'-{2,}', r'\n', text)
    # remove newline (inconsistency in separating different information)
    text = re.sub(r'\n{2,}', '; ', text)
    text = re.sub(r'\n', ' ', text)
    first_index, last_index = get_first_and_last_non_signal_index(text)
    if first_index:
        text = text[first_index:].strip()
    if last_index:
        text = text[:last_index+1].strip()
    
    return text

In [ ]:
print(text_list[4][980])

In [ ]:
text = text_list[3][120]
print(text)
text = clean_3(text)
print(text)

In [ ]:
def extract_and_replace(text, pattern):
    # Function to replace matched patterns with an empty string and extract the content
    def replacer(match):
        # Extract the text inside **...** and join parts with space if multiline
        extracted = ' '.join([part if part is not None else '' for part in match.groups()]).strip()
        extracted_texts.append(extracted)
        # Return an empty string to replace the matched pattern
        return ';'
    
    extracted_texts = []  # List to hold the extracted text
    modified_text = re.sub(pattern, replacer, text)  # Replace matched patterns with ''
    
    return extracted_texts, modified_text

In [ ]:
def extract_name_3(text):
    pattern = r'\*\*(.*?)\*\*(?:\n*\s*\*\*(.*?)\*\*)?'
    extracted_texts, modified_text = extract_and_replace(text, pattern)
    num = len(extracted_texts)
    modified_text = re.sub('\*', '', modified_text)
    return extracted_texts, modified_text, num


In [ ]:
names, modified_text, num = extract_name_3(text)
print("Extracted Names:", names)
# print("Modified Text:", modified_text)
print(num)
print(modified_text)


In [ ]:
# new: extract dates (use slices to be more efficient)
# to so: how to order the dates
def extract_dates_3(text, num):
    
    slices = text.split(';')
    slices = [s.strip() for s in slices if s]

    pattern = r'\b\d{1,2} [A-Z][a-z]{2} \d{4}\b'
    date_slice = [[]for s in slices]

    for i, s in enumerate(slices):
        dates = re.findall(pattern, s)
        slices[i] = ';' + re.sub(pattern, '', s) + ';'
        date_slice[i] = dates

    flattened_dates = [date for sublist in date_slice for date in sublist]
    date_list = flattened_dates

    assert num == int(len(date_list )/2), 'number of extracted dates not equal to number of names!'
    remaining_text = ';'.join(slices)
    
    return date_list, remaining_text

In [ ]:
date_list, remaining_text = extract_dates_3(modified_text, num)
print(date_list)
print(remaining_text)

In [ ]:
# new: extract ID info first
# two list: one is the list they are together, another the opposite. then combine them in right order.
def extract_ID_3(text, num):
    # id_list = []
    id_type_list = []
    id_number_list = []
    extract_id = 0
    
    slices = text.split(';')
    slices = [s.strip() for s in slices if s]
    # print(f'slices:{slices}')
    
    types = ['NRIC (Citizen)', 'Passport/Oth ers', 'ACRA Registration Number', 'FIN', 'NRIC (Permanent Resident)']
    pre_types = ['NRIC (C', 'Pass',  'ACRA', 'FIN', 'NRIC (P']
    # may have empty spaces which can not be extracted by full type name (if delete all empty spaces too early will obstruct the rest procedures)
    end_types = ['zen)', 'thers', 'mber', 'FIN', 'dent)']

    # reformat: separate id infos
    indexes = []
    add_indexes = []
    for t in types:
        escaped_type = re.escape(t)
        matches = re.finditer(escaped_type, text)
        for match in matches:
            indexes.append(match.start())  
            indexes.append(match.end())
    indexes = sorted(indexes)
    # print('indexes:')
    # print(indexes)
    for x in range(1,len(indexes)-1, 2):
        if x+1 < len(indexes) and indexes[x+1] - indexes[x] >2: # turning point (different sets of id info)
            add_indexes.append(indexes[x])
    if add_indexes:
        for idx in add_indexes:
            text_1 = text[:idx] + ";" + text[idx:]
        text = text_1
        slices = text.split(';')
        slices = [s.strip() for s in slices if s]

    # print(slices)

    # extract slice with id types and count type number
    type_dict = {key:value for key, value in zip(pre_types, types)}
    type_end_dict = {key:value for key, value in zip(pre_types, end_types)}
    types_pattern = '|'.join(re.escape(t) for t in pre_types)
    # pattern = rf'([0-9A-Z]+\s*({types_pattern}))'
    type_start_idx_list = [[] for i in slices]
    type_num_list = [[] for i in slices]

    # Q: what if different id info in one slice
    for i,s in enumerate(slices):
        match_iter = re.finditer(types_pattern , s)
        match_iter= list(match_iter)
        if match_iter:
            type_num = len(match_iter)
            type_num_list[i] = type_num
            type_start_idx = match_iter[0].start()
            type_start_idx_list[i] = type_start_idx

            # extract exact ID types
            text_after = s[type_start_idx:]
            id_types = []
            for m,match in enumerate(match_iter):
                pre_type = match.group()
                id_type = type_dict[pre_type]
                id_types.append(id_type)
                if m == type_num - 1:
                    type_ending = type_end_dict[pre_type]
                    idx = s[match.start():].find(type_ending)
                    if idx != -1:
                        idx = idx + len(type_ending)
                        text_after = s[match.start():][idx:] + ';'
                        # print(f'text_after:{text_after}')
                    else:
                        print(r'\nCannot find the full ID type (last one)!')
                        print(s)
                        print(f"Searching for type_ending: '{type_ending}' in text_after: {s[match.start():]}")

            id_type_list.append(' '.join(id_types))

            # extract Id number according to num of types and separate 
            text_ahead = s[:type_start_idx].strip()
            texts = re.split(r'\s+', text_ahead)
            # print(f'texts:\n{texts}')
            if type_num <= len(text):
                potential_id_number = texts[-type_num:]
                # print(f'potential_id_number:{potential_id_number}')
                if potential_id_number:
                    # check id numbers
                    pattern = re.compile(r'^[A-Z0-9]+$')
                    all_valid = all(pattern.match(item) for item in potential_id_number)
                    if all_valid:
                        id_number = ' '.join(potential_id_number).strip()
                        id_number_list.append(id_number)
                        text_ahead = ';' + ' '.join(texts[:len(texts)- type_num]) 
                        # print(f'text_ahead:{text_ahead}')
                    else:
                        print("Some extracted id numbers are not in the correct format:")
                        print(potential_id_number)
                

            else:
                print(f'Incomplete {i}th slice: {s}!')
    
            slices[i] = text_ahead + ' ' + text_after
            # print(f'{i}th slice: {slices[i]}')

        # no id type in this slice, but still may have id number
        # else:
            
    # print(id_number_list)
    # print(id_type_list)

    type_num_list = [i for i in type_num_list if i!=[]]
    remaining_text = ';'.join(slices)
    if len(id_number_list) == num:
        assert len(id_type_list) == num, 'Number of Id info not equal to number of names!'
        
    else:
        print('Likely ID number and Id type are not read together!')
        extract_id = 1
        

    return id_number_list, id_type_list, remaining_text, extract_id



In [ ]:
id_number_list, id_type_list, remaining_text, extract_id, type_num_list = extract_ID_3(modified_text, num)
print(id_number_list)
print(id_type_list)
print(remaining_text)


In [ ]:
# new: extract Nation and position

def extract_nation_posi_3(text, num):
    # appear only once (no mateer whether with multiple id infos)
    nation_list = []
    posi_list = []

    slices = text.split(';')
    slices = [s.strip() for s in slices if s]
    # print(slices)

    nation_posi_slice = [[] for s in slices]
    nation_slice = [[] for s in slices]
    posi_slice = [[] for s in slices]
    pattern = r'([A-Z]+)\s([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)'
    for i, s in enumerate(slices):
        nation_posis = re.findall(pattern, s)
        if nation_posis:
            slices[i] = ';' + re.sub(pattern, '', s) + ';'
            nation_posi_slice[i] = nation_posis
            
            nations, posis = zip(*nation_posis)
            nation_slice[i] = list(nations)
            posi_slice[i] = list(posis)
    
    nation_list = [nation for sublist in nation_slice for nation in sublist]
    posi_list = [posi for sublist in posi_slice for posi in sublist]
    # print(nation_list)
    # print(posi_list)

    assert len(nation_list) == len(posi_list) == num, 'number of extracted nationalities and positions not equal to number of names!'

    remaining_text = ';'.join(slices)
    return nation_list, posi_list, remaining_text

In [ ]:
nation_list, posi_list, remaining_text = extract_nation_posi_3(remaining_text, num)
print(nation_list)
print(posi_list)
print(remaining_text)

In [ ]:
# new: extract address (all got left)
# Q: whether need further check?
def extract_addr_3(text, num, extract_id):
    id_no_list = []
    slices = text.split(';')
    slices = [s.strip() for s in slices if s and any(char.isdigit() for char in s)]
    # print(slices)
    if extract_id == 0:
        assert len(slices) == num, 'number of slices not equal to number of names!'
        address_list = slices
    else:
        # extract address and ID number separately
        # texts = [s.split(' ') for s in slices]
        # len_text = [len(t) for t in texts]
        id_texts = [s for s in slices if len(s.split(' '))<5]
        address_list = [s for s in slices if len(s.split(' '))>=5]
        # id_num_list = [len(t.split(' ')) for t in id_texts]
        # assert id_num_list == type_num_list, 'Number of ID No. not equal to number of ID type!'
        id_no_list = id_texts

    return address_list, id_no_list

In [ ]:
address_list = extract_addr_3(remaining_text, num)
print(address_list)

In [ ]:

def organize_id_no(id_no_list_1, id_no_list_2, origin_text, num):
    id_no_list = []
    if id_no_list_2:
        id_no_list_combined = id_no_list_1 + id_no_list_2
        id_no_indexes = [origin_text.find(no) for no in id_no_list_combined]
        if -1 not in id_no_indexes:
            id_no_dict = {key:value for key,value in zip(id_no_indexes,id_no_list_combined)}
            id_indexes_sorted = id_no_indexes.copy()
            id_indexes_sorted.sort(reverse=False)
            for idx in id_indexes_sorted:
                id_no = id_no_dict[idx]
                id_no_list.append(id_no)    
        else:
            print('Cannot find several id number in the given text!')
    else:
        id_no_list = id_no_list_1
    
    assert len(id_no_list) == num, 'Number of ID No not equals to number of names!'
    
    return id_no_list
            
        
        

## CURRENT APPOINTMENT HOLDERS

In [ ]:
# new for part 3: name -> id -> dates -> nation, posi -> address
def curr_appt_hders(text):
    name_list = id_no_list = id_type_list = date_list = nation_list = posi_list = address_list = []
    
    modified_text = clean_3(text)
    # print(f'cleaned_text:\n{modified_text}')
    stripped_text = re.sub(r'[\*\s\;]', '', modified_text)
    # print(stripped_text)
    if len(stripped_text) < 10:
        name_list = id_no_list = id_type_list = date_list = nation_list = posi_list = address_list = ['-']
        print('Empty current apppointment holder!')
    else:
        origin_text = modified_text
        
        name_list, remaining_text, num = extract_name_3(modified_text)
        print(f'name_list:\n{name_list}\n')
        
        date_list, remaining_text = extract_dates_3(remaining_text, num)
        print(f'date_list:\n{date_list}\n')

        id_no_list_1, id_type_list, remaining_text, extract_id = extract_ID_3(remaining_text, num)
        print(f'id_type_list:\n{id_type_list}\n')


        nation_list, posi_list, remaining_text = extract_nation_posi_3(remaining_text, num)
        print(f'nation_list:\n{nation_list}\n') 
        print(f'posi_list:\n{posi_list}\n') 

        address_list, id_no_list_2 = extract_addr_3(remaining_text, num, extract_id)
        print(f'address_list:\n{address_list}\n') 

        id_no_list = organize_id_no(id_no_list_1, id_no_list_2, origin_text, num)
        print(f'id_no_list:\n{id_no_list}')
    
    # check the id_no_list and id_type list
    # example: original extracted text 0244381C NRIC S0244381C (Citizen) NRIC (Citizen)
    # normaly the id_type should be the same, maybe can just delete the id type in id_no_list
    

    return name_list, id_no_list, id_type_list, date_list, nation_list, posi_list, address_list

    

In [ ]:
idx = 87
text = text_list[3][idx]
# print(text,'\n')
print(file_list[idx], '\n')
name_list, id_number_list, id_type_list, date_list, nation_list, posi_list, address_list = curr_appt_hders(text)

